<a href="https://colab.research.google.com/github/explrA/Precog-Tasks/blob/main/Task2a.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Task 2.a:**

Phrase Similarity using Average of Token Embeddings

In [1]:
!pip install gensim
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.4 MB/s eta 0:00:00


In [ ]:
from huggingface_hub import login
login()

In [3]:
from datasets import load_dataset
from gensim.downloader import load
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

import random
import pandas as pd
import numpy as np
import re

In [4]:
ds = load_dataset("PiC/phrase_similarity")

word2vec_model = load('word2vec-google-news-300')


train_1 = ds['train']
dev_1 = ds['validation']
test_1 = ds['test']


train1_df = pd.DataFrame(train_1)
train_df = train1_df.drop(columns=['sentence1', 'sentence2', 'idx'])
# print(train_df)

dev1_df = pd.DataFrame(dev_1)
dev_df = dev1_df.drop(columns=['sentence1', 'sentence2', 'idx'])
# print(dev_df)

test1_df = pd.DataFrame(test_1)
test_df = test1_df.drop(columns=['sentence1', 'sentence2', 'idx'])
# print(test_df)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/5.47k [00:00<?, ?B/s]

phrase_similarity.py:   0%|          | 0.00/4.75k [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/1.42M [00:00<?, ?B/s]

PS-hard/validation/0000.parquet:   0%|          | 0.00/202k [00:00<?, ?B/s]

PS-hard/test/0000.parquet:   0%|          | 0.00/403k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7004 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [5]:
def mean_word_embeds(phrase, w2v_model):

  tokens = phrase.lower().split()

  word_embeds = []
  for word in tokens:
    if word in w2v_model:
      word_embeds.append(w2v_model[word])

  if not word_embeds:
    return np.zeros(w2v_model.vector_size)

  return np.mean(word_embeds, axis=0)



def cosine_sim(phrase1, phrase2, w2v_model):

  phrase_embed1 = mean_word_embeds(phrase1, w2v_model)
  phrase_embed2 = mean_word_embeds(phrase2, w2v_model)

  phrase_similarity = cosine_similarity([phrase_embed1], [phrase_embed2]) [0][0]
  # print(phrase_similarity)

  return phrase_similarity


In [6]:
train_df['cal_sim'] = train_df.apply(lambda i: cosine_sim(i['phrase1'], i['phrase2'], word2vec_model), axis=1)
train_sim = train_df['cal_sim'].values
train_labels = train_df['label'].values
train_sim_2d = np.array(train_sim).reshape(-1, 1)

dev_df['cal_sim'] = dev_df.apply(lambda j: cosine_sim(j['phrase1'], j['phrase2'], word2vec_model), axis=1)
dev_sim = dev_df['cal_sim'].values
dev_labels = dev_df['label'].values
dev_sim_2d = np.array(dev_sim).reshape(-1, 1)


classifier = LogisticRegression()
classifier.fit(train_sim_2d, train_labels)

dev_pred = classifier.predict(dev_sim_2d)
dev_acc = accuracy_score(dev_labels, dev_pred)
print("dev acc: ", dev_acc)


dev acc:  0.548


In [7]:
test_df['cal_sim'] = test_df.apply(lambda j: cosine_sim(j['phrase1'], j['phrase2'], word2vec_model), axis=1)
test_sim = test_df['cal_sim'].values
test_labels = test_df['label'].values
test_sim_2d = np.array(test_sim).reshape(-1, 1)

test_pred = classifier.predict(test_sim_2d)
test_acc = accuracy_score(test_labels, test_pred)
print("test_acc: ",test_acc)

test_acc:  0.5175
